# Подключим библиотеки и создадим экземпляр клиента для работы с GoogleAPI

In [ ]:
import apiclient.discovery
import httplib2
from oauth2client.service_account import ServiceAccountCredentials

CREDENTIALS_FILE = (
    ""  # Имя файла с закрытым ключом, вы должны подставить свое!!!
)

# Читаем ключи из файла
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    CREDENTIALS_FILE,
    [
        "https://www.googleapis.com/auth/spreadsheets",
        "https://www.googleapis.com/auth/drive",
    ],
)

httpAuth = credentials.authorize(httplib2.Http())  # Авторизуемся в системе
service = apiclient.discovery.build(
    "sheets", "v4", http=httpAuth
)  # Выбираем работу с таблицами и 4 версию API

spreadsheetId = "1anLwzOo3pKIL02JCteFPE21b8BKCdYY25zksFMVTIwg" # spreadsheetId из задания


# Основное задание
### 1. Сдвинем колонки "Годовой темп прироста" на 1 колонку вправо


In [ ]:
# Получаем существующие колонки
ranges = [f"{'Stats'}!{'D2:F10'}"]
results = (
    service.spreadsheets()
    .values()
    .batchGet(
        spreadsheetId=spreadsheetId,
        ranges=ranges,
        dateTimeRenderOption="FORMATTED_STRING",
    )
    .execute()
)
data = results["valueRanges"][0]["values"]
print(data)

# Вставляем их на 1 колонку правее

results = (
    service.spreadsheets()
    .values()
    .batchUpdate(
        spreadsheetId=spreadsheetId,
        body={
            "valueInputOption": "USER_ENTERED",
            "data": [
                {
                    "range": "Stats!E2:G10",
                    "majorDimension": "ROWS",
                    "values": data,
                }
            ],
        },
    )
    .execute()
)



### 2. Перекопируем заголовок "Годовой темп прироста"


In [ ]:
# Отмняем объединение ячеек D1:F1
results = (
    service.spreadsheets()
    .batchUpdate(
        spreadsheetId=spreadsheetId,
        body={
            "requests": [
                {
                    "unmergeCells": {
                        "range": {
                            "sheetId": 0,
                            "startRowIndex": 0,
                            "endRowIndex": 2,
                            "startColumnIndex": 3,
                            "endColumnIndex": 6,
                        }
                    }
                }
            ]
        },
    )
    .execute()
)

# Получим содержимое заголовка
ranges = [f"{'Stats'}!{'D1'}"]
results = (
    service.spreadsheets()
    .values()
    .batchGet(
        spreadsheetId=spreadsheetId,
        ranges=ranges,
        dateTimeRenderOption="FORMATTED_STRING",
    )
    .execute()
)
title = results["valueRanges"][0]["values"]

# Вставим заголовок на одну ячейку правее
results = (
    service.spreadsheets()
    .values()
    .batchUpdate(
        spreadsheetId=spreadsheetId,
        body={
            "valueInputOption": "USER_ENTERED",
            "data": [
                {
                    "range": "Stats!E1",
                    "majorDimension": "ROWS",
                    "values": title,
                }
            ],
        },
    )
    .execute()
)

# Объединим ячейки E1:H1
results = (
    service.spreadsheets()
    .batchUpdate(
        spreadsheetId=spreadsheetId,
        body={
            "requests": [
                {
                    "mergeCells": {
                        "range": {
                            "sheetId": 0,
                            "startRowIndex": 0,
                            "endRowIndex": 1,
                            "startColumnIndex": 4,
                            "endColumnIndex": 8,
                        }
                    }
                }
            ]
        },
    )
    .execute()
)


### 3. Дополняем новой колонкой "Прирост за месяц"

In [ ]:
# Данные из задания для колонки "Прирост за месяц"
data = [["01.06.2023", "353", "0", "6", "0", "-10", "218", "132", "8"]]

# Вставим эти данные на позицию D2:D10
results = (
    service.spreadsheets()
    .values()
    .batchUpdate(
        spreadsheetId=spreadsheetId,
        body={
            "valueInputOption": "USER_ENTERED",
            "data": [
                {
                    "range": "Stats!D2:D10",
                    "majorDimension": "COLUMNS",
                    "values": data,
                }
            ],
        },
    )
    .execute()
)

# Объединяем ячейки для заголовка "Прирост за месяц"
results = (
    service.spreadsheets()
    .batchUpdate(
        spreadsheetId=spreadsheetId,
        body={
            "requests": [
                {
                    "mergeCells": {
                        "range": {
                            "sheetId": 0,
                            "startRowIndex": 0,
                            "endRowIndex": 1,
                            "startColumnIndex": 1,
                            "endColumnIndex": 4,
                        }
                    }
                }
            ]
        },
    )
    .execute()
)


### 4. Дополняем 1й колокой "Годовой темп прироста"

In [ ]:
data = [["01.06.2023", "22,3", "0", "-0,7", "0", "73,1", "19,7", "10,6", "81,3"]]

results = (
    service.spreadsheets()
    .values()
    .batchUpdate(
        spreadsheetId=spreadsheetId,
        body={
            "valueInputOption": "USER_ENTERED",
            "data": [
                {
                    "range": "Stats!H2:H10",
                    "majorDimension": "COLUMNS",
                    "values": data,
                }
            ],
        },
    )
    .execute()
)
